In [12]:
import requests
import urllib.error
import urllib.parse
from urllib import request
import json
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import re
import time

In [13]:
df = pd.read_csv('op shop list only id.csv')

In [14]:
df['website'] = ''

In [15]:
def read_the_url(urllink):
    res = requests.get(urllink)
    js = json.loads(res.text)
    return js


def test_token(js):
    try:
        token = js['next_page_token']
    except KeyError:
        return True ### there is no token
    else:
        return False ### there is a token
    
    
def get_the_place_id(url):
    list_of_name=[]
    place_id_list=[]
    js = read_the_url(url)
    
    for i in range(1,4):
        if test_token(js) is True:
            for item in js['results']:
                list_of_name.append(item['name'])
                place_id_list.append(item['place_id'])
            break
        else:
            token = js['next_page_token']
            URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken=" + token + "&key=" #add your key
            for item in js['results']:
                list_of_name.append(item['name'])
                place_id_list.append(item['place_id'])
            time.sleep(2)
            js = read_the_url(URL)
        
    df = pd.DataFrame(list_of_name, columns=['name'])
    df = pd.concat([df, pd.DataFrame(place_id_list,columns=['place_id'])],axis=1)
    
    return df


def get_street_address_and_suburb(detail):
    street_address = ''
    suburb_info = ''
    street_number = ''
    route = ''
    suburb = ''
    for item in detail['result']['address_components']:
        if item['types'] == ['street_number']:
            street_number = item['long_name']
        
        elif item['types'] == ['route']:
            route = item['long_name']
            
        elif item['types'] == ['locality', 'political']:
            suburb = item['long_name']
            
    street_address = street_number + ' '+ route
    suburb_info = suburb
    
    return street_address, suburb_info

In [16]:
for i in range(len(df)):
    place_id = df['place_id'][i]
    URL_of_place_id = "https://maps.googleapis.com/maps/api/place/details/json?place_id=" + place_id + "&key=" #add your key
    detail = read_the_url(URL_of_place_id)
    df['address'][i] = detail['result']['formatted_address']
    df['lat'][i] = detail['result']['geometry']['location']['lat']
    df['lng'][i] = detail['result']['geometry']['location']['lng']
    street, suburb = get_street_address_and_suburb(detail)
     
    try:
        df['phone_number'][i] =  detail['result']['formatted_phone_number']
    except KeyError:
        df['phone_number'][i] = 'unknown'
    else:
        df['phone_number'][i] = detail['result']['formatted_phone_number']
        
    try:
        df['website'][i] =  detail['result']['website']
    except KeyError:
        df['website'][i] = 'No website available'
    else:
        df['website'][i] = detail['result']['website']
        
    try:
        df['rating'][i] =  detail['result']['rating']
    except KeyError:
        df['rating'][i] = 'no rating'
    else:
        df['rating'][i] = detail['result']['rating']
        
    try:
        detail['result']['opening_hours']['weekday_text']
    except KeyError:
        df['Monday'][i] = 'please call for detail'
        df['Tuesday'][i] = 'please call for detail'
        df['Wednesday'][i] = 'please call for detail'
        df['Thursday'][i] = 'please call for detail'
        df['Friday'][i] = 'please call for detail'
        df['Saturday'][i] = 'please call for detail'
        df['Sunday'][i] = 'please call for detail'
    else:
        df['Monday'][i] = detail['result']['opening_hours']['weekday_text'][0].replace(' – ',' to ')
        df['Tuesday'][i] = detail['result']['opening_hours']['weekday_text'][1].replace(' – ',' to ')
        df['Wednesday'][i] = detail['result']['opening_hours']['weekday_text'][2].replace(' – ',' to ')
        df['Thursday'][i] = detail['result']['opening_hours']['weekday_text'][3].replace(' – ',' to ')
        df['Friday'][i] = detail['result']['opening_hours']['weekday_text'][4].replace(' – ',' to ')
        df['Saturday'][i] = detail['result']['opening_hours']['weekday_text'][5].replace(' – ',' to ')
        df['Sunday'][i] = detail['result']['opening_hours']['weekday_text'][6].replace(' – ',' to ')
    
    df['street_address'][i] = street
    df['suburb'][i] = suburb
        


In [17]:
for i in range(len(df)):
    df['Monday'][i] = df['Monday'][i].replace('Monday:','')
    df['Tuesday'][i] = df['Tuesday'][i].replace('Tuesday:','')
    df['Wednesday'][i] = df['Wednesday'][i].replace('Wednesday:','')
    df['Thursday'][i] = df['Thursday'][i].replace('Thursday:','')
    df['Friday'][i] = df['Friday'][i].replace('Friday:','')
    df['Saturday'][i] = df['Saturday'][i].replace('Saturday:','')
    df['Sunday'][i] = df['Sunday'][i].replace('Sunday:','')

In [18]:
df.to_csv('final op shop list.csv')

In [19]:
jsonlist = []
for i in range(len(df)):
    aItem = {}
    aItem["id"] = i
    aItem["name"] = df['name'][i]
    aItem["address"] = df['address'][i]
    aItem['lat'] = df['lat'][i]
    aItem['lng'] = df['lng'][i]
    aItem['street'] = df['street_address'][i]
    aItem['suburb'] = df['suburb'][i]
    aItem['phone'] = df ['phone_number'][i]
    aItem['Monday'] = df['Monday'][i]
    aItem['Tuesday'] = df['Tuesday'][i]
    aItem['Wednesday'] = df['Wednesday'][i]
    aItem['Thursday'] = df['Thursday'][i]
    aItem['Friday'] = df['Friday'][i]
    aItem['Saturday'] = df['Saturday'][i]
    aItem['Sunday'] = df['Sunday'][i]
    
    aItem['website'] = df['website'][i]
    aJson = json.dumps(aItem)
    jsonlist.append(aItem)

In [20]:
savefile = 'final op shop list.json'

with open(savefile, 'w') as f:
    f.write(json.dumps(jsonlist))